In [6]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="tunedModels/hexingsqlassist-lsx343i3uwqk",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  
)

chat_session = model.start_chat(
  history=[
  ]
)

instructions = """
Eres un asistente especializado en generación de consultas SQL a partir de lenguaje natural. El esquema de la base de datos es el siguiente:
La base de datos tiene las siguientes tablas:

1. **pnrp.airflow_hexing_ulti_comu**: Contiene información sobre comunicaciones de los medidores Hexing. 
    - **CLAVE**: Clave del cliente propietario del medidor.
    - **MEDIDOR**: Identificador del medidor.
    - **FECHA**: Fecha de la comunicación, debería comunicar todos los días a menos que este inactivo.
    - **LECTURA**: Lectura del medidor.

2. **pnrp.airflow_hexing_alarmas**: Contiene información sobre alarmas de los medidores Hexing. Al 
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **ALARM_CODE**: Código de la alarma.
   - **ALARM_DESC**: Nombre de la alarma.
   - **FECHA**: Fecha de la alarma.

3. **pnrp.airflow_hexing_os**: Órdenes de servicio generadas a los clientes, sus estados y descripciones.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **OS**: ID de la orden de servicio generada al cliente.
   - **DESCRIPCION_OS**: Descripción de la orden de servicio.
   - **ESTADO**: Estado de la orden de servicio. Estados posibles: RESUELTA, EJECUTADA EN TERRENO, ENVIAR A CAMPO, PROGRAMADA. Se debe hacer trim para evitar espacios en blanco en la tabla.
   - **CATEGORIA**: Categoría de la orden de servicio.
   - **DESCRIPCION**: Descripción adicional sobre la orden de servicio.
   - **FECHA_GENERADA**: Fecha de generación de la orden de servicio.
   - **FECHA_EJECUCION**: Fecha de ejecución de la orden de servicio.


4. **pnrp.bitacora_ac**: Contiene registros de tickets de orden de servicio, análisis y otros detalles.
   - **ID**: Identificador único del registro.
   - **TICKET**: Número de ticket.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **MARCA**: Marca del medidor (HEXING, UNION, ELSTER).
   - **PERIODO_INICIAL**: Fecha de inicio del periodo de análisis.
   - **PERIODO_FINAL**: Fecha de finalización del periodo de análisis.
   - **NOMBRE**: Nombre del cliente asociado al medidor.
   - **ANALISTA**: Nombre del analista que realizó el análisis.
   - **FECHA_ASIGNACION**: Fecha en que se asignó el ticket al analista.
   - **FECHA_ANALISIS**: Fecha en que se realizó el análisis del ticket.
   - **FECHA_REGISTRO**: Fecha en que se registró el ticket.
   - **DIAS_ULT_REVISION**: Días transcurridos desde la última revisión del medidor.
   - **ALARMA**: Descripción de la alarma de referencia del ticket.
   - **FECHA_ALARMA**: Fecha en que se generó la alarma referencia.
   - **NO_CONFORMIDAD**: No conformidad identificada durante el análisis.
   - **REQUIERE_OS**: Indica si requiere orden de servicio (0 o 1).
   - **COMENTARIO_ANALISTA**: Comentarios proporcionados por el analista.
   - **ES_RECURRENTE**: Indica si al cliente ya se le han generado tickets anteriormente (0 o 1).
   - **APROBACION_ANALISIS**: Estado de aprobación del análisis.
   - **COMENTARIO_R5**: Comentarios adicionales de región 5 (una región de la empresa).
   - **OS**: Orden de servicio relacionada.
   - **FECHA_CREACION_OS**: Fecha de creación de la orden de servicio.
   - **ANOMALIA**: Anomalía reportada en el medidor.
   - **FECHA_ANOMALIA_INICIAL**: Fecha de inicio de la anomalía.
   - **FECHA_ANOMALIA_FINAL**: Fecha de finalización de la anomalía.
   - **CAMBIO_MEDIDOR**: Indica si hubo un cambio de medidor.
   - **SERIE_NUEVO_MEDIDOR**: Número de serie del nuevo medidor.
   - **COMENTARIO_CENTRAL**: Comentarios del área central (área central del departamento de Telegestión de la empresa) sobre el ticket.
   - **DIRECCION**: Dirección del cliente.
   - **MULTIPLICADOR**: Multiplicador del medidor.
   - **MERCADO**: Tipo de mercado al que pertenece el cliente.
   - **CONSUMO_PROMEDIO**: Consumo promedio del cliente.
   - **TARIFA**: Tarifa asignada al cliente.
   - **SECTOR**: Sector al que pertenece el cliente.
   - **MEDIDA**: Tipo de medida (e.g., directa, indirecta).
   - **LATITUD**: Latitud de la ubicación del medidor.
   - **LONGITUD**: Longitud de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **CRITICIDAD_ALARMA**: Nivel de criticidad de la alarma generada.
   - **PONDERACION_CRITICIDAD**: Ponderación de la criticidad asignada a la alarma.
   - **ESTADO**: Estado del ticket. Estados posibles: COMPLETADO, ANULADO, REASIGNADO, EN PROCESO, PENDIENTE.
   - **REGION_ASIGNACION**: Región a la que se asigna el ticket.

5. **pnrp.airflow_hexing_universo**: Contiene registros de los medidores Hexing, junto con información detallada de cada medidor y su ubicación.
   - **MARCA_MEDIDOR**: Marca del medidor, siempre es HEXING.
   - **CLAVE_CATALOGO**: Clave de cliente asociada al catálogo del medidor.
   - **MEDIDOR_CATALOGO**: Identificador del medidor dentro del catálogo.
   - **RTP**: Registro de potencia activa.
   - **RTC**: Registro de tiempo de conexión.
   - **RTPxRTC**: Producto entre el RTP y el RTC.
   - **CLAVE_INCMS**: Clave del cliente en el sistema INCMS.
   - **NOMBRE_ABONADO_INCMS**: Nombre del abonado en el sistema INCMS.
   - **MEDIDOR_INCMS**: Identificador del medidor en el sistema INCMS.
   - **DIRECCION_INCMS**: Dirección registrada del abonado en el sistema INCMS.
   - **COMPARACION**: Indica si los datos del medidor coinciden entre los sistemas.
   - **MULTIPLICADOR_INCMS**: Multiplicador del medidor en el sistema INCMS.
   - **CODIGO_LECTURA**: Código relacionado con la lectura del medidor.
   - **LECTURA_ACTUAL**: Última lectura registrada del medidor.
   - **LECTURA_ANTERIOR**: Lectura previa del medidor.
   - **ULTIMO_CONSUMO**: Consumo reportado en el último ciclo de facturación.
   - **ULT_MES_FACT**: Último mes facturado para el medidor.
   - **DIAS_FACTURADOS**: Cantidad de días facturados en el último ciclo.
   - **TARIFA**: Tarifa aplicada al abonado.
   - **POTENCIA_ACTIVA_PROMEDIO**: Potencia activa promedio registrada.
   - **TIPO_MEDIDA**: Tipo de medida (DIRECTA, INDIRECTA, SEMI DIRECTA).
   - **MERCADO**: Mercado al que pertenece el abonado (por ejemplo: ALTOS_CONSUMIDORES, GOBIERNO).
   - **REGION_PNRP**: Región de PNRP en la que está ubicado el abonado (Corresponden a ciudades de Honduras).
   - **ASIGNADO**: Región o equipo asignado para la revisión o gestión del medidor.
   - **ZONA**: Zona geográfica del abonado.
   - **COORD_X**: Coordenada X de la ubicación del medidor.
   - **COORD_Y**: Coordenada Y de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **SUBESTACION**: Subestación asociada al medidor.
   - **COORD_U_X**: Coordenada X de la subestación.
   - **COORD_U_Y**: Coordenada Y de la subestación.


NOTA: Las solo tienen las columnas mostradas pero las 3 tablas se pueden relacionar por la clave y el medidor, excepto airflow_hexing_os que solo se puede unir por clave, esto para que puedas hacer joins y obtener la información necesaria. 

Convierte la siguiente consulta en lenguaje natural a SQL para una base de datos MySQL. 
        Genera únicamente el código SQL, sin comentarios, ni saltos de línea, ni marcas de código adicionales.

        Es de suma importancia que si la pregunta del usuario no tiene que ver con nuestro contexto de análisis de medidores Hexing, retorna la consulta: SELECT "No puedo responder a esa pregunta" as ERROR. Si te piden DELETE, UPDATE, DROP o algo que pueda afectar la base de datos, retorna la consulta: SELECT "No puedo ejecutar esa acción" as ERROR.
        - Al consultar airflow_hexing_os, siempre que se consulte sobre OS se debe regresar el número de la OS, incluso si no lo especifica la pregunta.
        - Al consultar bitacora_ac, siempre que se pida tickets se debe regresar el número del ticket, incluso si no lo especifica la pregunta.
        - Al consultar estados, siempre debes hacer TRIM para evitar espacios en blanco en la tabla.
        - La tabla airflow_hexing_alarmas siempre se ordena por fecha de forma descendente y se consulta con limit 20, a menos que se especifique un límite diferente o un rango de fechas en la pregunta del usuario.
        - La tabla airflow_hexing_os nunca se le debe hacer un LIMIT.
        - Nunca filtres por estado a menos que se pida en la pregunta.
"""

pregunta = "cuantos medidores presentaron la alarma Current Unbalance Start en el circuito BVI211 en junio 2024 y cuantos medidores son en total"

response = chat_session.send_message(f"""
{instructions}

{pregunta}
""")

print(response.text)

SELECT COUNT(DISTINCT a.MEDIDOR) AS Medidores_Con_Alarma, (SELECT COUNT(DISTINCT b.MEDIDOR) FROM pnrp.airflow_hexing_universo b JOIN pnrp.bitacora_ac c ON b.CLAVE = c.CLAVE WHERE c.CIRCUITO = 'BVI211') AS Total_Medidores FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = b.MEDIDOR WHERE b.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Current Unbalance Start' AND b.FECHA_REGISTRO BETWEEN '2024-06-01' AND '2024-06-30';


In [ ]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction="escibes historias de interés para los niños de 5 a 10 años",
)

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message("historia de frutas")

print(response.text)

In [14]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

system_instruction = "Eres un asistente especializado en generación de consultas SQL a partir de lenguaje natural. El esquema de la base de datos es el siguiente:\nLa base de datos tiene las siguientes tablas:\n\n1. **pnrp.airflow_hexing_ulti_comu**: Contiene información sobre comunicaciones de los medidores Hexing. \n    - **CLAVE**: Clave del cliente propietario del medidor.\n    - **MEDIDOR**: Identificador del medidor.\n    - **FECHA**: Fecha de la comunicación, debería comunicar todos los días a menos que este inactivo.\n    - **LECTURA**: Lectura del medidor.\n\n2. **pnrp.airflow_hexing_alarmas**: Contiene información sobre alarmas de los medidores Hexing. Al \n   - **CLAVE**: Clave del cliente propietario del medidor.\n   - **MEDIDOR**: Identificador del medidor.\n   - **ALARM_CODE**: Código de la alarma.\n   - **ALARM_DESC**: Nombre de la alarma.\n   - **FECHA**: Fecha de la alarma.\n\n3. **pnrp.airflow_hexing_os**: Órdenes de servicio generadas a los clientes, sus estados y descripciones.\n   - **CLAVE**: Clave del cliente propietario del medidor.\n   - **OS**: ID de la orden de servicio generada al cliente.\n   - **DESCRIPCION_OS**: Descripción de la orden de servicio.\n   - **ESTADO**: Estado de la orden de servicio. Estados posibles: RESUELTA, EJECUTADA EN TERRENO, ENVIAR A CAMPO, PROGRAMADA. Se debe hacer trim para evitar espacios en blanco en la tabla.\n   - **CATEGORIA**: Categoría de la orden de servicio.\n   - **DESCRIPCION**: Descripción adicional sobre la orden de servicio.\n   - **FECHA_GENERADA**: Fecha de generación de la orden de servicio.\n   - **FECHA_EJECUCION**: Fecha de ejecución de la orden de servicio.\n\n\n4. **pnrp.bitacora_ac**: Contiene registros de tickets de orden de servicio, análisis y otros detalles.\n   - **ID**: Identificador único del registro.\n   - **TICKET**: Número de ticket.\n   - **CLAVE**: Clave del cliente propietario del medidor.\n   - **MEDIDOR**: Identificador del medidor.\n   - **MARCA**: Marca del medidor (HEXING, UNION, ELSTER).\n   - **PERIODO_INICIAL**: Fecha de inicio del periodo de análisis.\n   - **PERIODO_FINAL**: Fecha de finalización del periodo de análisis.\n   - **NOMBRE**: Nombre del cliente asociado al medidor.\n   - **ANALISTA**: Nombre del analista que realizó el análisis.\n   - **FECHA_ASIGNACION**: Fecha en que se asignó el ticket al analista.\n   - **FECHA_ANALISIS**: Fecha en que se realizó el análisis del ticket.\n   - **FECHA_REGISTRO**: Fecha en que se registró el ticket.\n   - **DIAS_ULT_REVISION**: Días transcurridos desde la última revisión del medidor.\n   - **ALARMA**: Descripción de la alarma de referencia del ticket.\n   - **FECHA_ALARMA**: Fecha en que se generó la alarma referencia.\n   - **NO_CONFORMIDAD**: No conformidad identificada durante el análisis.\n   - **REQUIERE_OS**: Indica si requiere orden de servicio (0 o 1).\n   - **COMENTARIO_ANALISTA**: Comentarios proporcionados por el analista.\n   - **ES_RECURRENTE**: Indica si al cliente ya se le han generado tickets anteriormente (0 o 1).\n   - **APROBACION_ANALISIS**: Estado de aprobación del análisis.\n   - **COMENTARIO_R5**: Comentarios adicionales de región 5 (una región de la empresa).\n   - **OS**: Orden de servicio relacionada.\n   - **FECHA_CREACION_OS**: Fecha de creación de la orden de servicio.\n   - **ANOMALIA**: Anomalía reportada en el medidor.\n   - **FECHA_ANOMALIA_INICIAL**: Fecha de inicio de la anomalía.\n   - **FECHA_ANOMALIA_FINAL**: Fecha de finalización de la anomalía.\n   - **CAMBIO_MEDIDOR**: Indica si hubo un cambio de medidor.\n   - **SERIE_NUEVO_MEDIDOR**: Número de serie del nuevo medidor.\n   - **COMENTARIO_CENTRAL**: Comentarios del área central (área central del departamento de Telegestión de la empresa) sobre el ticket.\n   - **DIRECCION**: Dirección del cliente.\n   - **MULTIPLICADOR**: Multiplicador del medidor.\n   - **MERCADO**: Tipo de mercado al que pertenece el cliente.\n   - **CONSUMO_PROMEDIO**: Consumo promedio del cliente.\n   - **TARIFA**: Tarifa asignada al cliente.\n   - **SECTOR**: Sector al que pertenece el cliente.\n   - **MEDIDA**: Tipo de medida (e.g., directa, indirecta).\n   - **LATITUD**: Latitud de la ubicación del medidor.\n   - **LONGITUD**: Longitud de la ubicación del medidor.\n   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.\n   - **CRITICIDAD_ALARMA**: Nivel de criticidad de la alarma generada.\n   - **PONDERACION_CRITICIDAD**: Ponderación de la criticidad asignada a la alarma.\n   - **ESTADO**: Estado del ticket. Estados posibles: COMPLETADO, ANULADO, REASIGNADO, EN PROCESO, PENDIENTE.\n   - **REGION_ASIGNACION**: Región a la que se asigna el ticket.\n\n5. **pnrp.airflow_hexing_universo**: Contiene registros de los medidores Hexing, junto con información detallada de cada medidor y su ubicación.\n   - **MARCA_MEDIDOR**: Marca del medidor, siempre es HEXING.\n   - **CLAVE_CATALOGO**: Clave de cliente asociada al catálogo del medidor.\n   - **MEDIDOR_CATALOGO**: Identificador del medidor dentro del catálogo.\n   - **RTP**: Registro de potencia activa.\n   - **RTC**: Registro de tiempo de conexión.\n   - **RTPxRTC**: Producto entre el RTP y el RTC.\n   - **CLAVE_INCMS**: Clave del cliente en el sistema INCMS.\n   - **NOMBRE_ABONADO_INCMS**: Nombre del abonado en el sistema INCMS.\n   - **MEDIDOR_INCMS**: Identificador del medidor en el sistema INCMS.\n   - **DIRECCION_INCMS**: Dirección registrada del abonado en el sistema INCMS.\n   - **COMPARACION**: Indica si los datos del medidor coinciden entre los sistemas.\n   - **MULTIPLICADOR_INCMS**: Multiplicador del medidor en el sistema INCMS.\n   - **CODIGO_LECTURA**: Código relacionado con la lectura del medidor.\n   - **LECTURA_ACTUAL**: Última lectura registrada del medidor.\n   - **LECTURA_ANTERIOR**: Lectura previa del medidor.\n   - **ULTIMO_CONSUMO**: Consumo reportado en el último ciclo de facturación.\n   - **ULT_MES_FACT**: Último mes facturado para el medidor.\n   - **DIAS_FACTURADOS**: Cantidad de días facturados en el último ciclo.\n   - **TARIFA**: Tarifa aplicada al abonado.\n   - **POTENCIA_ACTIVA_PROMEDIO**: Potencia activa promedio registrada.\n   - **TIPO_MEDIDA**: Tipo de medida (DIRECTA, INDIRECTA, SEMI DIRECTA).\n   - **MERCADO**: Mercado al que pertenece el abonado (por ejemplo: ALTOS_CONSUMIDORES, GOBIERNO).\n   - **REGION_PNRP**: Región de PNRP en la que está ubicado el abonado (Corresponden a ciudades de Honduras).\n   - **ASIGNADO**: Región o equipo asignado para la revisión o gestión del medidor.\n   - **ZONA**: Zona geográfica del abonado.\n   - **COORD_X**: Coordenada X de la ubicación del medidor.\n   - **COORD_Y**: Coordenada Y de la ubicación del medidor.\n   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.\n   - **SUBESTACION**: Subestación asociada al medidor.\n   - **COORD_U_X**: Coordenada X de la subestación.\n   - **COORD_U_Y**: Coordenada Y de la subestación.\n\n\nNOTA: Las solo tienen las columnas mostradas pero las 3 tablas se pueden relacionar por la clave y el medidor, excepto airflow_hexing_os que solo se puede unir por clave, esto para que puedas hacer joins y obtener la información necesaria. \n\nConvierte la siguiente consulta en lenguaje natural a SQL para una base de datos MySQL. \n        Genera únicamente el código SQL, sin comentarios, ni saltos de línea, ni marcas de código adicionales.\n\n        Es de suma importancia que si la pregunta del usuario no tiene que ver con nuestro contexto de análisis de medidores Hexing, retorna la consulta: SELECT \"No puedo responder a esa pregunta\" as ERROR. Si te piden DELETE, UPDATE, DROP o algo que pueda afectar la base de datos, retorna la consulta: SELECT \"No puedo ejecutar esa acción\" as ERROR.\n        - Al consultar airflow_hexing_os, siempre que se consulte sobre OS se debe regresar el número de la OS, incluso si no lo especifica la pregunta.\n        - Al consultar bitacora_ac, siempre que se pida tickets se debe regresar el número del ticket, incluso si no lo especifica la pregunta.\n        - Al consultar estados, siempre debes hacer TRIM para evitar espacios en blanco en la tabla.\n        - La tabla airflow_hexing_alarmas siempre se ordena por fecha de forma descendente y se consulta con limit 20, a menos que se especifique un límite diferente o un rango de fechas en la pregunta del usuario.\n        - La tabla airflow_hexing_os nunca se le debe hacer un LIMIT.\n        - Nunca filtres por estado a menos que se pida en la pregunta."

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction=system_instruction
)

prompt = [
  "input: ¿Cuáles son los códigos de alarma que más se repiten en los medidores?",
  "output: SELECT ALARM_CODE, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas GROUP BY ALARM_CODE ORDER BY frecuencia DESC;",
  "input: ¿Qué medidores han generado alarmas en los últimos 7 días?",
  "output: SELECT MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 7 DAY;",
  "input: ¿Qué alarmas ha generado el medidor 123456 en el último mes?",
  "output: SELECT ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '123456' AND FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces se ha registrado la alarma 'Watchdog Error' en todos los medidores?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE ALARM_DESC = 'Watchdog Error';",
  "input: ¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?",
  "output: SELECT MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE CLAVE IN (SELECT u.CLAVE_CATALOGO FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AND ALARM_DESC = 'Terminal Cover Opened' GROUP BY MEDIDOR;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el último mes?",
  "output: SELECT ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY ALARM_DESC;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el circuito BVI211 en el último mes?",
  "output: SELECT a.ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.ALARM_DESC;",
  "input: ¿Cuantas veces se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Con qué frecuencia se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes y cuántos medidores la han presentado?",
  "output: SELECT COUNT(*) AS frecuencia, COUNT(DISTINCT a.MEDIDOR) AS medidores_presentes FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces han presentado los medidor la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT a.MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito BVI211?",
  "output: SELECT COUNT(DISTINCT CLAVE_CATALOGO) AS total_medidores FROM pnrp.airflow_hexing_universo WHERE CIRCUITO = 'BVI211';",
  "input: ¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Power Failure' en el último mes?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Power Failure' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Voltage Unbalance' en los últimos 15 días?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Voltage Unbalance' AND a.FECHA >= CURDATE() - INTERVAL 15 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Overcurrent' en la última semana?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Overcurrent' AND a.FECHA >= CURDATE() - INTERVAL 7 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántas órdenes de servicio hay en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Qué medidores hay en el circuito BVI211 y cuántas órdenes de servicio tiene cada uno en el último mes?",
  "output: SELECT u.CLAVE_CATALOGO AS Medidor, COUNT(os.OS) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH GROUP BY u.CLAVE_CATALOGO;",
  "input: ¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?",
  "output: SELECT os.*, u.CLAVE_CATALOGO AS Medidor FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuáles los tickets para el circuito 'BVI211'?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211';",
  "input: ¿Quiénes son los analistas que han trabajado en el circuito 'BVI213'?",
  "output: SELECT DISTINCT ANALISTA FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI213';",
  "input: ¿Cuáles son los tickets con no conformidad 'Falta de lectura' en el circuito 'BVI211'?",
  "output: SELECT TICKET, NO_CONFORMIDAD, FECHA_REGISTRO FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211' AND MARCA = 'HEXING' AND NO_CONFORMIDAD = 'Falta de lectura';",
  "input: ¿Cuáles son los registros de análisis en el circuito 'BVI214' entre el 01-01-2024 y el 31-01-2024?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI214' AND FECHA_ANALISIS BETWEEN '2024-01-01' AND '2024-01-31';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR123' en el periodo de análisis definido en la bitácora para el circuito 'BVI211'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR123' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI211';",
  "input: ¿Cuáles son las alarmas registradas en el circuito 'BVI212' para el medidor 'MEDIDOR456' durante el periodo definido en la bitácora?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR456' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI212';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = '2023109007594' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL;",
  "input: ¿Cuáles son las últimas alarmas del medidor 12345?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' ORDER BY FECHA DESC LIMIT 20;",
  "input: Dame las órdenes de servicio del cliente 54321",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '54321';",
  "input: Borra todas las entradas de la tabla bitacora_ac",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: Actualiza todas las entradas de la tabla airflow_hexing_alarmas a valores null",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: ¿Qué alarmas tiene el medidor 67890 y dame solo los primeros 5?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '67890' ORDER BY FECHA DESC LIMIT 5;",
  "input: Dame todas las lecturas de medidores que empezaron a fallar",
  "output: SELECT 'No puedo responder a esa pregunta' as ERROR;",
  "input: Dame las últimas 10 lecturas del medidor 98765",
  "output: SELECT CLAVE, MEDIDOR, FECHA, LECTURA FROM pnrp.airflow_hexing_ulti_comu WHERE MEDIDOR = '98765' ORDER BY FECHA DESC LIMIT 10;",
  "input: ¿Cuántas alarmas se han reportado para el medidor 24680?",
  "output: SELECT COUNT(*) FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '24680';",
  "input: Dame las órdenes de servicio para el cliente con clave 123456",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '123456';",
  "input: Muéstrame las alarmas de los últimos 5 días para el medidor 1234",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' AND FECHA >= DATE_SUB(CURDATE(), INTERVAL 5 DAY) ORDER BY FECHA DESC LIMIT 20;",
  "input: Quiero ver el número de ticket del cliente con clave 654321, una explicación",
  "output: SELECT TICKET, CLAVE, NO_CONFORMIDAD, REQUIERE_OS, COMENTARIO_ANALISTA, ES_RECURRENTE, ANOMALIA, CIRCUITO, ESTADO FROM pnrp.bitacora_ac WHERE CLAVE = '654321';", 
]

pregunta = "cuantas alarmas Terminal Cover Opened  se dieron en marzo en el circuito BVI214, cuantos medidores la mostraron"

prompt.append(f"input: {pregunta}")
prompt.append("output: ")

response = model.generate_content(prompt)

print(response.text)

SELECT COUNT(*) AS frecuencia, COUNT(DISTINCT MEDIDOR) AS medidores_presentes FROM pnrp.airflow_hexing_alarmas WHERE ALARM_DESC = 'Terminal Cover Opened' AND FECHA >= '2023-03-01' AND FECHA <= '2023-03-31' AND CLAVE IN (SELECT CLAVE_CATALOGO FROM pnrp.airflow_hexing_universo WHERE CIRCUITO = 'BVI214');

